In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk, Label, Entry, Button, OptionMenu, StringVar

class GeneticAlgorithm:
    def __init__(self, pop_size=1000, dimensions=10, min_val=-5.12, max_val=5.12, num_generations=1000, mutation_rate=0.1, crossover_rate=0.8):
        self.pop_size = pop_size
        self.dimensions = dimensions
        self.min_val = min_val
        self.max_val = max_val
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

    def rastrigin(self, x):
        A = 10
        return A * self.dimensions + np.sum(x**2 - A * np.cos(2 * np.pi * x))

    def initialize_population(self):
        return np.random.uniform(low=self.min_val, high=self.max_val, size=(self.pop_size, self.dimensions))

    @staticmethod
    def one_point_crossover(parents):
        crossover_point = np.random.randint(1, parents.shape[1])
        offspring = np.empty_like(parents)

        for i in range(parents.shape[0]):
            parent1_idx = i % parents.shape[0]
            parent2_idx = (i+1) % parents.shape[0]
            offspring[i, :crossover_point] = parents[parent1_idx, :crossover_point]
            offspring[i, crossover_point:] = parents[parent2_idx, crossover_point:]

        return offspring

    @staticmethod
    def two_point_crossover(parents):
        crossover_points = sorted(np.random.choice(parents.shape[1], 2, replace=False))
        offspring = np.empty_like(parents)

        for i in range(parents.shape[0]):
            parent1_idx = i % parents.shape[0]
            parent2_idx = (i+1) % parents.shape[0]
            offspring[i, :crossover_points[0]] = parents[parent1_idx, :crossover_points[0]]
            offspring[i, crossover_points[0]:crossover_points[1]] = parents[parent2_idx, crossover_points[0]:crossover_points[1]]
            offspring[i, crossover_points[1]:] = parents[parent1_idx, crossover_points[1]:]

        return offspring

    def n_point_crossover(self, parents, n=2):
        if n >= self.dimensions:
            n = self.dimensions - 1  # Ensure n is within valid range
        crossover_points = np.sort(np.random.choice(self.dimensions - 1, n, replace=False))
        offspring = np.empty_like(parents)

        for i in range(parents.shape[0]):
            for j in range(n):
                if j % 2 == 0:
                    start = crossover_points[j]
                    end = crossover_points[j + 1] if j + 1 < n else self.dimensions
                    offspring[i, start:end] = parents[i % 2, start:end]
                else:
                    start = crossover_points[j]
                    end = crossover_points[j + 1] if j + 1 < n else self.dimensions
                    offspring[i, start:end] = parents[(i + 1) % 2, start:end]

        return offspring

    def uniform_mutation(self, offspring):
        mask = np.random.random(size=offspring.shape) < self.mutation_rate
        mutation_amounts = np.random.uniform(low=-0.1, high=0.1, size=offspring.shape)
        offspring += mask * mutation_amounts
        return offspring

    def gaussian_mutation(self, offspring):
        mask = np.random.random(size=offspring.shape) < self.mutation_rate
        mutation_amounts = np.random.normal(loc=0, scale=0.1, size=offspring.shape)
        offspring += mask * mutation_amounts
        return offspring

    def bitflip_mutation(self, offspring):
        mask = np.random.random(size=offspring.shape) < self.mutation_rate
        offspring[mask] = 1 - offspring[mask]
        return offspring

    def roulette_wheel_selection(self, population, fitness):
        probabilities = fitness / np.sum(fitness)
        selected_indices = np.random.choice(len(fitness), size=len(fitness), p=probabilities)
        return population[selected_indices], selected_indices

    def tournament_selection(self, population, fitness, tournament_size=2):
        selected_indices = []
        for _ in range(len(fitness)):
            tournament_indices = np.random.choice(len(fitness), size=tournament_size, replace=False)
            tournament_fitness = fitness[tournament_indices]
            selected_indices.append(tournament_indices[np.argmin(tournament_fitness)])
        return population[selected_indices], selected_indices

    def evolve(self, mutation_method, crossover_method, parent_selection_method, survivor_selection_method, run):
        population = self.initialize_population()
        best_fitnesses = []
        selected_indices_history = []

        for generation in range(self.num_generations):
            fitness = np.apply_along_axis(self.rastrigin, 1, population)
            best_fitness = np.min(fitness)
            best_fitnesses.append(best_fitness)

            parents, selected_indices = parent_selection_method(population, fitness)
            offspring_crossover = crossover_method(parents)
            offspring_mutation = mutation_method(offspring_crossover)

            population = survivor_selection_method(population, offspring_mutation, fitness)
            selected_indices_history.append(selected_indices)

            # Print only run number and best fitness
            print(f"Run: {run+1}, Generation: {generation + 1}, Best Fitness: {best_fitness}")

        return best_fitnesses, selected_indices_history

def copy_population(population, offspring, fitness):
    return np.copy(offspring)

class DifferentialEvolution:
    def __init__(self, pop_size=100, dimensions=10, min_val=-5.12, max_val=5.12, num_generations=100):
        self.pop_size = pop_size
        self.dimensions = dimensions
        self.min_val = min_val
        self.max_val = max_val
        self.num_generations = num_generations

    def rastrigin(self, x):
        A = 10
        return A * self.dimensions + np.sum(x**2 - A * np.cos(2 * np.pi * x))

    def initialize_population(self):
        population = np.zeros((self.pop_size, self.dimensions))
        for i in range(self.dimensions):
            population[:, i] = np.random.uniform(i / self.dimensions, (i + 1) / self.dimensions, self.pop_size)
        population = self.min_val + population * (self.max_val - self.min_val)
        return population

    def mutation(self, population):
        indices = np.random.choice(len(population), size=3, replace=False)
        mutant_vector = population[indices[0]] + 0.5 * (population[indices[1]] - population[indices[2]])
        return np.clip(mutant_vector, self.min_val, self.max_val)

    def crossover(self, target_vector, mutant_vector):
        crossover_mask = np.random.rand(self.dimensions) < 0.9
        return np.where(crossover_mask, mutant_vector, target_vector)

    def evolve(self, num_runs):
        overall_best_solution = None
        overall_best_fitness = float('inf')
        best_fitnesses = []
        generation_fitnesses = []

        for run in range(num_runs):
            population = self.initialize_population()
            best_solution = None
            best_fitness = float('inf')
            generation_fitness = []

            for generation in range(self.num_generations):
                for target_idx, target_vector in enumerate(population):
                    mutant_vector = self.mutation(population)
                    trial_vector = self.crossover(target_vector, mutant_vector)
                    trial_fitness = self.rastrigin(trial_vector)
                    target_fitness = self.rastrigin(target_vector)

                    if trial_fitness < target_fitness:
                        population[target_idx] = trial_vector
                        if trial_fitness < best_fitness:
                            best_fitness = trial_fitness
                            best_solution = trial_vector

                if best_fitness < overall_best_fitness:
                    overall_best_fitness = best_fitness
                    overall_best_solution = best_solution

                best_fitnesses.append(best_fitness)
                generation_fitness.append(best_fitness)
                print(f"Run: {run + 1}, Generation: {generation + 1}, Best Fitness: {best_fitness}")

            generation_fitnesses.append(generation_fitness)

        print("Overall Best Solution:", overall_best_solution)
        print("Overall Best Fitness:", overall_best_fitness)

        # Plotting the fitness values for each generation
        plt.figure()
        for i, fitnesses in enumerate(generation_fitnesses):
            plt.plot(range(1, self.num_generations + 1), fitnesses, label=f"Run {i + 1}")
        plt.xlabel('Generation')
        plt.ylabel('Best Fitness')
        plt.title('Best Fitness over Generations')
        plt.legend()
        plt.show()

class GeneticAlgorithmGUI:
    def __init__(self, master):
        self.master = master
        master.title("Genetic Algorithm")

        self.selection_label = Label(master, text="Select Algorithm:")
        self.selection_label.grid(row=0, column=0)
        self.algorithm_var = StringVar(master)
        self.algorithm_var.set("Genetic Algorithm")
        self.algorithm_menu = OptionMenu(master, self.algorithm_var, "Genetic Algorithm", "Differential Evolution", command=self.update_options)
        self.algorithm_menu.grid(row=0, column=1)

        self.method_label = Label(master, text="Select Method:")
        self.method_label.grid(row=1, column=0)
        self.method_var = StringVar(master)
        self.method_var.set("Select")
        self.method_menu = OptionMenu(master, self.method_var, *["Select", "One-Point Crossover", "Two-Point Crossover", "N-Point Crossover"])
        self.method_menu.grid(row=1, column=1)

        self.pop_size_label = Label(master, text="Population Size:")
        self.pop_size_label.grid(row=2, column=0)
        self.pop_size_entry = Entry(master)
        self.pop_size_entry.grid(row=2, column=1)

        self.dimensions_label = Label(master, text="Dimensions:")
        self.dimensions_label.grid(row=3, column=0)
        self.dimensions_entry = Entry(master)
        self.dimensions_entry.grid(row=3, column=1)

        self.min_val_label = Label(master, text="Minimum Value:")
        self.min_val_label.grid(row=4, column=0)
        self.min_val_entry = Entry(master)
        self.min_val_entry.grid(row=4, column=1)

        self.max_val_label = Label(master, text="Maximum Value:")
        self.max_val_label.grid(row=5, column=0)
        self.max_val_entry = Entry(master)
        self.max_val_entry.grid(row=5, column=1)

        self.num_generations_label = Label(master, text="Number of Generations:")
        self.num_generations_label.grid(row=6, column=0)
        self.num_generations_entry = Entry(master)
        self.num_generations_entry.grid(row=6, column=1)

        self.num_runs_label = Label(master, text="Number of Runs:")
        self.num_runs_label.grid(row=7, column=0)
        self.num_runs_entry = Entry(master)
        self.num_runs_entry.grid(row=7, column=1)

        self.run_button = Button(master, text="Run Algorithm", command=self.run_algorithm)
        self.run_button.grid(row=8, columnspan=2)

    def update_options(self, *args):
        selected_algorithm = self.algorithm_var.get()
        if selected_algorithm == "Genetic Algorithm":
            self.method_menu['menu'].delete(0, 'end')
            for method in ["Select", "One-Point Crossover", "Two-Point Crossover", "N-Point Crossover"]:
                self.method_menu['menu'].add_command(label=method, command=lambda value=method: self.method_var.set(value))
        elif selected_algorithm == "Differential Evolution":
            # You can add options for Differential Evolution here if needed
            pass

    def run_algorithm(self):
        algorithm = self.algorithm_var.get()
        if algorithm == "Genetic Algorithm":
            pop_size = int(self.pop_size_entry.get())
            dimensions = int(self.dimensions_entry.get())
            min_val = float(self.min_val_entry.get())
            max_val = float(self.max_val_entry.get())
            num_generations = int(self.num_generations_entry.get())
            num_runs = int(self.num_runs_entry.get())

            selected_method = self.method_var.get()
            if selected_method == "Select":
                print("Please select a method.")
                return

            ga = GeneticAlgorithm(pop_size, dimensions, min_val, max_val, num_generations)
            mutation_method = getattr(ga, selected_method.lower().replace('-', '_') + '_mutation')
            if mutation_method is None:
                print("Invalid mutation method selected.")
                return
            ga.evolve(mutation_method, num_runs)
        elif algorithm == "Differential Evolution":
            # Add code to run Differential Evolution here
            pass

if __name__ == "__main__":
    root = Tk()
    app = GeneticAlgorithmGUI(root)
    root.mainloop()
